# Features acteurs oscar

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from CinePred.data.featuring import *
from CinePred.data.preprocessing import *
from CinePred.data.importing import *

# Aller on taff

In [2]:
acteurs_df = pd.read_csv('../raw_data/cat_acteur.csv')


In [3]:
acteurs_df

,Unnamed: 0,title,shifted,acteur_name,year,month,budget,income,ratio
0,0,#Selfieparty,0,Ksenia Mishina,2016,3,100000.0,179458,1.794580
1,1,#Selfieparty,0,Pavlo Li,2016,3,100000.0,179458,1.794580
2,2,#Selfieparty,0,Olena Lavrenyuk,2016,3,100000.0,179458,1.794580
3,3,'Chi chi' de ai,0,Dee Hsu,2017,5,1500000.0,4013033,2.675355
4,4,'Chi chi' de ai,0,Kim Scar,2017,5,1500000.0,4013033,2.675355
...,...,...,...,...,...,...,...,...,...
37096,37096,Öldür beni,0,Burak Sarimola,2009,2,63657.0,19142,0.300705
37097,37097,Öldür beni,0,Erol Alpsoykan,2009,2,63657.0,19142,0.300705
37098,37098,Únos,0,Ludovít Cittel,2017,3,1504620.0,1633936,1.085946
37099,37099,Únos,0,Gabriela Dolná,2017,3,1504620.0,1633936,1.085946


In [5]:
df = preprocess_example(path = "../raw_data/IMDb_movies.csv")

----- import Data -----
----- keep columns -----
----- remove na rows -----
----- convert budget -----
----- reduce column type -----
----- convert income column -----
----- convert to int -----
----- convert to date -----
----- log transform -----
----- data_shape -----


In [10]:
df = add_success_movies_per_actors(df, path = "../raw_data/cat_acteur.csv")

AttributeError: 'NoneType' object has no attribute 'merge'

In [12]:
df = pd.read_csv('../raw_data/cat_acteur.csv')

In [13]:
df

,Unnamed: 0,title,shifted,acteur_name,year,month,budget,income,ratio
0,0,#Selfieparty,0,Ksenia Mishina,2016,3,100000.0,179458,1.794580
1,1,#Selfieparty,0,Pavlo Li,2016,3,100000.0,179458,1.794580
2,2,#Selfieparty,0,Olena Lavrenyuk,2016,3,100000.0,179458,1.794580
3,3,'Chi chi' de ai,0,Dee Hsu,2017,5,1500000.0,4013033,2.675355
4,4,'Chi chi' de ai,0,Kim Scar,2017,5,1500000.0,4013033,2.675355
...,...,...,...,...,...,...,...,...,...
37096,37096,Öldür beni,0,Burak Sarimola,2009,2,63657.0,19142,0.300705
37097,37097,Öldür beni,0,Erol Alpsoykan,2009,2,63657.0,19142,0.300705
37098,37098,Únos,0,Ludovít Cittel,2017,3,1504620.0,1633936,1.085946
37099,37099,Únos,0,Gabriela Dolná,2017,3,1504620.0,1633936,1.085946


## On définit le ratio de chaque films

In [4]:
acteurs_df['ratio'] = acteurs_df['income']/acteurs_df['budget']

### Détermination des catégories des acteurs

In [5]:
acteurs_df_cat = acteurs_df.loc[(acteurs_df['ratio']>=5) & (acteurs_df['budget'] >= 100_000_000)]

In [6]:
len(acteurs_df_cat['acteur_name'].unique())

159

In [7]:
acteurs_df_cat[['acteur_name','budget']].groupby(by='acteur_name').sum().sort_values(by="budget", ascending=False).head(159)

,budget
acteur_name,
Robert Downey Jr.,1.522000e+09
Mark Ruffalo,1.147000e+09
Chris Evans,5.760000e+08
Chris Hemsworth,5.710000e+08
Tom Hanks,5.250000e+08
...,...
Tom Hardy,1.000000e+08
George Clooney,1.000000e+08
Fionn Whitehead,1.000000e+08


### Catégorisation

In [8]:
top_tier_actor_df = acteurs_df.groupby(by="acteur_name").sum()[['budget']].sort_values("budget", ascending=False)

In [9]:
top_tier_actor_df.head(50)

,budget
acteur_name,
Mac,4.043730e+09
Johnny Depp,3.407500e+09
Robert Downey Jr.,3.175500e+09
Leon,2.895191e+09
Will Smith,2.653100e+09
Bruce Willis,2.619495e+09
Tom Cruise,2.604100e+09
Ali,2.482064e+09
Robert De Niro,2.362104e+09


In [10]:
def connupaconnu(X, ratio = 3):
    if (X['budget'] >= 100_000_000) & (X['ratio'] >= 3):
        return 1
    return 0

In [11]:
acteurs_df.sort_values('year', ascending=True, inplace= True)

In [12]:
acteurs_df['connu'] = acteurs_df.apply(connupaconnu, axis = 1)

In [13]:
acteurs_df['nbsucces'] = acteurs_df['connu']

In [14]:
new_df = acteurs_df

In [15]:
new_df['nbsuccess'] = acteurs_df.groupby(by  ='acteur_name')['connu'].cumsum(axis = 0)

In [16]:
new_df = new_df.sort_values('year', ascending=True)

In [17]:
new_df.drop(columns='nbsucces', inplace = True)

In [18]:
new_df['connu2'] = new_df['nbsuccess'].apply(lambda x : 1 if x >=1 else 0)

In [19]:
new_df.drop(columns="connu", inplace = True)

In [20]:
new_df.rename(columns={"connu2" : "connu"}, inplace=True)

In [21]:
new_df[new_df['title'].str.contains('Star Trek')]

,Unnamed: 0,title,shifted,acteur_name,year,month,budget,income,ratio,nbsuccess,connu
29822,29822,Star Trek,1,Kelley,1979,4,35000000.0,82604699,2.360134,0,0
29823,29823,Star Trek,1,DeForest Kelley,1979,4,35000000.0,82604699,2.360134,0,0
29826,29826,Star Trek,1,Leon,1979,4,35000000.0,82604699,2.360134,0,0
29825,29825,Star Trek,1,William Shatner,1979,4,35000000.0,82604699,2.360134,0,0
29824,29824,Star Trek,1,Leonard Nimoy,1979,4,35000000.0,82604699,2.360134,0,0
29840,29840,Star Trek II - L'ira di Khan,0,William Shatner,1982,10,11200000.0,78912963,7.045800,0,0
29839,29839,Star Trek II - L'ira di Khan,0,Leon,1982,10,11200000.0,78912963,7.045800,0,0
29838,29838,Star Trek II - L'ira di Khan,0,Kelley,1982,10,11200000.0,78912963,7.045800,0,0
29837,29837,Star Trek II - L'ira di Khan,0,DeForest Kelley,1982,10,11200000.0,78912963,7.045800,0,0
29841,29841,Star Trek II - L'ira di Khan,0,Leonard Nimoy,1982,10,11200000.0,78912963,7.045800,0,0


In [22]:
new_df.drop(index=new_df[new_df['acteur_name'] == "Leon" ].index)

,Unnamed: 0,title,shifted,acteur_name,year,month,budget,income,ratio,nbsuccess,connu
13749,13749,Il gabinetto del dottor Caligari,0,Werner Krauss,1920,2,1.800000e+04,8811,0.489500,0,0
13748,13748,Il gabinetto del dottor Caligari,0,Conrad Veidt,1920,2,1.800000e+04,8811,0.489500,0,0
13750,13750,Il gabinetto del dottor Caligari,0,Friedrich Feher,1920,2,1.800000e+04,8811,0.489500,0,0
21464,21464,Markens grøde,0,Amund Rydland,1921,12,2.926274e+04,4272,0.145988,0,0
21463,21463,Markens grøde,0,Karen Poulsen,1921,12,2.926274e+04,4272,0.145988,0,0
...,...,...,...,...,...,...,...,...,...,...,...
17805,17805,La Marca del Demonio,0,Eivaut Rischen,2020,1,9.000000e+05,396624,0.440693,0,0
9712,9712,Figli,0,Paola Cortellesi,2020,1,2.314800e+06,3691574,1.594770,0,0
10015,10015,Force of Nature,0,Emile Hirsch,2020,6,2.300000e+07,537474,0.023368,0,0
32875,32875,The Tax Collector,3,Cinthya Carmona,2020,8,3.000000e+07,942666,0.031422,0,0


In [23]:
liste_fi = new_df[['acteur_name','title','year','nbsuccess','connu']].values.tolist()

In [24]:
new_df['totalsuccess'] = new_df.groupby(by = 'title').cumsum()['nbsuccess']

In [25]:
# new_df['totalsuccess2'] = new_df.groupby(by = 'title')['totalsuccess']


# df.groupby(['Mt'], sort=False)['count'].max()

In [26]:
total_success = pd.DataFrame(new_df.groupby(['title'], sort = False)['totalsuccess'].max())

In [27]:
total_success.reset_index(inplace=True)

In [28]:
total_success[total_success['title'].str.contains('Harry')]

,title,totalsuccess
387,"Give 'em Hell, Harry!",0
1454,"Harry, ti presento Sally...",0
2540,Harry a pezzi,0
3302,Harry Potter e la pietra filosofale,3
3435,Come Harry divenne un albero,0
3794,Harry Potter e la camera dei segreti,3
4260,Harry Potter e il prigioniero di Azkaban,4
4734,Harry Potter e il calice di fuoco,3
5773,Harry Potter e l'Ordine della Fenice,5
6736,I'm Not Harry Jenson.,0


In [29]:
total_success.sort_values('totalsuccess', ascending=False).head(50)

,title,totalsuccess
11715,Avengers: Endgame,19
11480,Avengers: Infinity War,16
11612,Fast & Furious - Hobbs & Shaw,14
10026,Revenant - Redivivo,13
10766,Pirati dei Caraibi - La vendetta di Salazar,13
10619,Captain America: Civil War,13
11674,C'era una volta a... Hollywood,13
11072,Guardiani della Galassia Vol. 2,12
9604,Lo Hobbit - La battaglia delle cinque armate,11
11533,Jumanji - The Next Level,11


# Chercher l'acteur qu'on veut (optionnel)

In [30]:
acteurs_df[acteurs_df['title'].str.contains("La maschera di ferro")].sort_values('budget', ascending=False).sort_values('year')

,Unnamed: 0,title,shifted,acteur_name,year,month,budget,income,ratio,connu,nbsucces,nbsuccess
18556,18556,La maschera di ferro,2,Leon,1998,3,35000000.0,182968902,5.227683,0,0,1
18555,18555,La maschera di ferro,2,Leonardo DiCaprio,1998,3,35000000.0,182968902,5.227683,0,0,1
18554,18554,La maschera di ferro,2,Jeremy Irons,1998,3,35000000.0,182968902,5.227683,0,0,0
18553,18553,La maschera di ferro,2,John Malkovich,1998,3,35000000.0,182968902,5.227683,0,0,0


In [31]:
acteurs_df.drop(columns='nbsucces', inplace = True)

In [32]:
acteurs_df[acteurs_df['acteur_name'].str.contains("Leonardo DiCaprio")]
acteurs_df['shifted'] = acteurs_df['nbsuccess'] - acteurs_df['connu']

In [33]:
acteurs_df[['acteur_name','title','shifted']].groupby(by="title")

In [34]:
acteurs_df

,Unnamed: 0,title,shifted,acteur_name,year,month,budget,income,ratio,connu,nbsuccess
13749,13749,Il gabinetto del dottor Caligari,0,Werner Krauss,1920,2,1.800000e+04,8811,0.489500,0,0
13748,13748,Il gabinetto del dottor Caligari,0,Conrad Veidt,1920,2,1.800000e+04,8811,0.489500,0,0
13750,13750,Il gabinetto del dottor Caligari,0,Friedrich Feher,1920,2,1.800000e+04,8811,0.489500,0,0
21464,21464,Markens grøde,0,Amund Rydland,1921,12,2.926274e+04,4272,0.145988,0,0
21463,21463,Markens grøde,0,Karen Poulsen,1921,12,2.926274e+04,4272,0.145988,0,0
...,...,...,...,...,...,...,...,...,...,...,...
33430,33430,Tobulas Pasimatymas,5,Leon,2020,1,2.314800e+05,947455,4.093032,0,5
6905,6905,Cut Throat City,0,Eiza González,2020,8,8.000000e+06,507974,0.063497,0,0
6904,6904,Cut Throat City,0,Ethan Hawke,2020,8,8.000000e+06,507974,0.063497,0,0
24583,24583,Padre no hay más que uno 2: La llegada de la s...,0,Santiago Segura,2020,7,4.700000e+06,11700000,2.489362,0,0


In [35]:
total_success = pd.DataFrame(acteurs_df.groupby(['title'], sort = False)[['acteur_name','shifted']].max())
total_success.reset_index(inplace = True)

In [36]:
total_success[total_success['acteur_name'].str.contains('Leonardo DiCaprio')]

,title,acteur_name,shifted
1913,Buon compleanno Mr. Grape,Leonardo DiCaprio,0
2374,Romeo + Giulietta di William Shakespeare,Leonardo DiCaprio,0
2565,Titanic,Leonardo DiCaprio,0
2796,La maschera di ferro,Leonardo DiCaprio,1
3837,Gangs of New York,Leonardo DiCaprio,1
4314,The Aviator,Leonardo DiCaprio,1
5177,Blood Diamond - Diamanti di sangue,Leonardo DiCaprio,1
6207,Revolutionary Road,Leonardo DiCaprio,1
7458,Inception,Leonardo DiCaprio,1
8082,J. Edgar,Leonardo DiCaprio,2


In [37]:
acteurs_df[acteurs_df['acteur_name'].str.contains('Margot Robbie')]

,Unnamed: 0,title,shifted,acteur_name,year,month,budget,income,ratio,connu,nbsuccess
33081,33081,The Wolf of Wall Street,0,Margot Robbie,2013,1,100000000.0,392000694,3.920007,1,1
30241,30241,Suite Francese,1,Margot Robbie,2014,3,17361000.0,9337930,0.537868,0,1
29422,29422,Sopravvissuti,1,Margot Robbie,2015,8,7500000.0,381839,0.050912,0,1
9968,9968,Focus - Niente è come sembra,1,Margot Robbie,2015,3,50100000.0,158762963,3.168921,0,1
36215,36215,Whiskey Tango Foxtrot,1,Margot Robbie,2016,5,35000000.0,24972139,0.713490,0,1
33520,33520,Tonya,1,Margot Robbie,2017,3,11000000.0,53939297,4.903572,0,1
4361,4361,Bombshell - La voce dello scandalo,1,Margot Robbie,2019,4,32000000.0,59863792,1.870743,0,1
4921,4921,C'era una volta a... Hollywood,1,Margot Robbie,2019,9,90000000.0,374343626,4.159374,0,1
3970,3970,Birds of Prey e la fantasmagorica rinascita di...,1,Margot Robbie,2020,2,84500000.0,201858461,2.388858,0,1


In [38]:
last_df = pd.DataFrame(acteurs_df.groupby('title').sum()["shifted"])

In [39]:
last_df.reset_index(inplace = True)

In [40]:
last_df[last_df['title'].str.contains('Avengers')]

,title,shifted
952,Avengers: Age of Ultron,7
953,Avengers: Endgame,16
954,Avengers: Infinity War,13
9968,The Avengers,4
9969,The Avengers - Agenti speciali,0


In [41]:
acteurs_df

,Unnamed: 0,title,shifted,acteur_name,year,month,budget,income,ratio,connu,nbsuccess
13749,13749,Il gabinetto del dottor Caligari,0,Werner Krauss,1920,2,1.800000e+04,8811,0.489500,0,0
13748,13748,Il gabinetto del dottor Caligari,0,Conrad Veidt,1920,2,1.800000e+04,8811,0.489500,0,0
13750,13750,Il gabinetto del dottor Caligari,0,Friedrich Feher,1920,2,1.800000e+04,8811,0.489500,0,0
21464,21464,Markens grøde,0,Amund Rydland,1921,12,2.926274e+04,4272,0.145988,0,0
21463,21463,Markens grøde,0,Karen Poulsen,1921,12,2.926274e+04,4272,0.145988,0,0
...,...,...,...,...,...,...,...,...,...,...,...
33430,33430,Tobulas Pasimatymas,5,Leon,2020,1,2.314800e+05,947455,4.093032,0,5
6905,6905,Cut Throat City,0,Eiza González,2020,8,8.000000e+06,507974,0.063497,0,0
6904,6904,Cut Throat City,0,Ethan Hawke,2020,8,8.000000e+06,507974,0.063497,0,0
24583,24583,Padre no hay más que uno 2: La llegada de la s...,0,Santiago Segura,2020,7,4.700000e+06,11700000,2.489362,0,0


In [42]:
laaast = pd.DataFrame(last_df.merge(acteurs_df, on='title').drop(columns=["connu","nbsuccess","shifted_y","Unnamed: 0"])).drop_duplicates()

In [43]:
# last_df.to_csv('../raw_data/cat_acteur.csv')

In [44]:
laaast[laaast['title'].str.contains('Avengers: Endgame')]

,title,shifted_x,acteur_name,year,month,budget,income,ratio
2959,Avengers: Endgame,16,Mark Ruffalo,2019,4,356000000.0,2797800564,7.85899
2960,Avengers: Endgame,16,Chris Evans,2019,4,356000000.0,2797800564,7.85899
2961,Avengers: Endgame,16,Robert Downey Jr.,2019,4,356000000.0,2797800564,7.85899


In [45]:
laaast.rename(columns={"shifted_x":'shifted'}, inplace=True)

In [46]:
# laaast.to_csv('../raw_data/cat_acteur.csv')

In [47]:
print('----- import Data -----')
df = import_data("../raw_data/IMDb movies.csv")

print('----- keep columns -----')
df = keep_columns(df,
                  column_names=[
                      'imdb_title_id', 'title', 'year', 'date_published',
                      'genre', 'duration', 'country', 'director', 'writer',
                      'production_company', 'actors', 'budget',
                      'worlwide_gross_income'
                  ])

print('----- remove na rows -----')
df = remove_na_rows(df)

print('----- convert budget -----')
df['budget'] = convert_budget_column(df[['budget']])

# print('----- reduce column type -----')
# df['actors'] = reduce_column_type(df[['actors']], nb_max=3)

print('----- convert income column -----')
df['worlwide_gross_income'] = convert_income(df[['worlwide_gross_income']])

print('----- convert to int -----')
df['year'] = convert_to_int(df[['year']])
df['duration'] = convert_to_int(df[['duration']])

print('----- convert to date -----')
df['date_published'] = convert_to_date(df[['date_published']])

print('----- reset index -----')
df = reset_index(df)

# print('----- log transform -----')
# df['worlwide_gross_income'] = log_transformation(
#     df[['worlwide_gross_income']])

# print('----- data_shape -----')

----- import Data -----


/tmp/ipykernel_410/3978816577.py:2: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  df = import_data("../raw_data/IMDb movies.csv")


----- keep columns -----
----- remove na rows -----
----- convert budget -----
----- convert income column -----
----- convert to int -----
----- convert to date -----
----- reset index -----


In [48]:
df

,imdb_title_id,title,year,date_published,genre,duration,country,director,writer,production_company,actors,budget,worlwide_gross_income
0,tt0010323,Il gabinetto del dottor Caligari,1920,1920-02-27,"Fantasy, Horror, Mystery",76,Germany,Robert Wiene,"Carl Mayer, Hans Janowitz",Decla-Bioscop AG,"Werner Krauss, Conrad Veidt, Friedrich Feher, ...",18000,8811
1,tt0011440,Markens grøde,1921,1921-12-02,Drama,107,Norway,Gunnar Sommerfeldt,"Knut Hamsun, Gunnar Sommerfeldt",Christiana Film,"Amund Rydland, Karen Poulsen, Ragna Wettergree...",27887,4272
2,tt0012190,I quattro cavalieri dell'Apocalisse,1921,1923-04-16,"Drama, Romance, War",150,USA,Rex Ingram,"Vicente Blasco Ibáñez, June Mathis",Metro Pictures Corporation,"Pomeroy Cannon, Josef Swickard, Bridgetta Clar...",800000,9183673
3,tt0012349,Il monello,1921,1923-11-26,"Comedy, Drama, Family",68,USA,Charles Chaplin,Charles Chaplin,Charles Chaplin Productions,"Carl Miller, Edna Purviance, Jackie Coogan, Ch...",250000,26916
4,tt0014624,La donna di Parigi,1923,1927-06-06,"Drama, Romance",82,USA,Charles Chaplin,Charles Chaplin,Charles Chaplin Productions,"Edna Purviance, Clarence Geldart, Carl Miller,...",351000,11233
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12565,tt9878242,Subharathri,2019,2019-07-06,"Drama, Romance",130,India,Vyasan K.P.,"Vyasan K.P., Vyasan K.P.",Abaam Movies,"Dileep, Anu Sithara, Siddique, Suraj Venjaramo...",402722,105932
12566,tt9886872,Munthiri Monchan,2019,2019-12-06,"Comedy, Romance",130,India,Vijith Nambiar,"Manu Gopal, Meharali Poilungal Ismail",Vishvas Movies,"Manesh Krishnan, Gopika Anil, Salim Kumar, Dev...",402722,2568
12567,tt9894394,Upin & Ipin: Keris Siamang Tunggal,2019,2019-03-21,Animation,100,Malaysia,"Adam Bin Amiruddin, Syed Nurfaiz Khalid bin Sy...","Hjh. Ainon Ariff, Nur Naquyah bt Burhanuddin",Les' Copaque Production,"Asyiela Putri Bt Azhar, Mohd Amir Asyraf Bin M...",4729812,83857
12568,tt9900782,Kaithi,2019,2019-10-25,"Action, Thriller",145,India,Lokesh Kanagaraj,"Lokesh Kanagaraj, Pon Parthiban",Dream Warrior Pictures,"Karthi, Narain, Ramana, George Maryan, Harish ...",3221779,524061


In [49]:
df = add_success_movies_per_actors(df)

In [50]:
df

,imdb_title_id,title,year,date_published,genre,duration,country,director,writer,production_company,actors,budget,worlwide_gross_income,shifted
0,tt0010323,Il gabinetto del dottor Caligari,1920,1920-02-27,"Fantasy, Horror, Mystery",76,Germany,Robert Wiene,"Carl Mayer, Hans Janowitz",Decla-Bioscop AG,"Werner Krauss, Conrad Veidt, Friedrich Feher, ...",18000,8811,0
1,tt0011440,Markens grøde,1921,1921-12-02,Drama,107,Norway,Gunnar Sommerfeldt,"Knut Hamsun, Gunnar Sommerfeldt",Christiana Film,"Amund Rydland, Karen Poulsen, Ragna Wettergree...",27887,4272,0
2,tt0012190,I quattro cavalieri dell'Apocalisse,1921,1923-04-16,"Drama, Romance, War",150,USA,Rex Ingram,"Vicente Blasco Ibáñez, June Mathis",Metro Pictures Corporation,"Pomeroy Cannon, Josef Swickard, Bridgetta Clar...",800000,9183673,0
3,tt0012349,Il monello,1921,1923-11-26,"Comedy, Drama, Family",68,USA,Charles Chaplin,Charles Chaplin,Charles Chaplin Productions,"Carl Miller, Edna Purviance, Jackie Coogan, Ch...",250000,26916,0
4,tt0014624,La donna di Parigi,1923,1927-06-06,"Drama, Romance",82,USA,Charles Chaplin,Charles Chaplin,Charles Chaplin Productions,"Edna Purviance, Clarence Geldart, Carl Miller,...",351000,11233,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12021,tt10308928,Force of Nature,2020,2020-06-30,"Action, Drama",91,USA,Michael Polish,Cory M. Miller,Emmett/Furla/Oasis Films (EFO Films),"Emile Hirsch, Mel Gibson, David Zayas, Kate Bo...",23000000,537474,0
12022,tt10937670,Figli,2020,2020-01-23,Comedy,97,Italy,Giuseppe Bonito,Mattia Torre,Wildside,"Valerio Mastandrea, Paola Cortellesi, Stefano ...",2241574,3691574,0
12023,tt9086228,Gretel e Hansel,2020,2020-08-19,"Fantasy, Horror, Mystery",87,"Canada, USA, Ireland, South Africa",Oz Perkins,Rob Hayes,Orion Pictures,"Sophia Lillis, Samuel Leakey, Alice Krige, Jes...",5000000,21966625,2
12024,tt10251538,10 jours sans maman,2020,2020-02-19,Comedy,104,France,Ludovic Bernard,"Ariel Winograd, Mariano Vera",Soyouz Films,"Aure Atika, Franck Dubosc, Alexis Michalik, Hé...",1452363,10186502,0


In [51]:
df[df['actors'].str.contains('Leonardo DiCaprio')]

,imdb_title_id,title,year,date_published,genre,duration,country,director,writer,production_company,actors,budget,worlwide_gross_income,shifted
378,tt1343092,Il grande Gatsby,2013,2013-05-16,"Drama, Romance",143,"Australia, USA",Baz Luhrmann,"Baz Luhrmann, Craig Pearce",Warner Bros.,"Lisa Adam, Frank Aldridge, Amitabh Bachchan, S...",105000000,353641895,0
1870,tt0105156,La mia peggiore amica,1992,1992-05-08,"Drama, Thriller",90,USA,Katt Shea,"Melissa Goddard, Andy Ruben",New Line Cinema,"Sara Gilbert, Drew Barrymore, Tom Skerritt, Ch...",3000000,1829804,0
1927,tt0108550,Buon compleanno Mr. Grape,1993,1994-03-04,Drama,118,USA,Lasse Hallström,"Peter Hedges, Peter Hedges",Paramount Pictures,"Johnny Depp, Leonardo DiCaprio, Juliette Lewis...",11000000,10032765,0
2224,tt0114214,Pronti a morire,1995,1995-05-26,"Action, Romance, Thriller",107,"Japan, USA",Sam Raimi,Simon Moore,TriStar Pictures,"Sharon Stone, Gene Hackman, Russell Crowe, Leo...",32000000,18636537,0
2275,tt0112461,Ritorno dal nulla,1995,1995-04-21,"Biography, Crime, Drama",102,USA,Scott Kalvert,"Jim Carroll, Bryan Goluboff",New Line Cinema,"Leonardo DiCaprio, Lorraine Bracco, Marilyn So...",2000000,2381087,0
2319,tt0114702,Poeti dall'inferno,1995,1996-08-23,"Biography, Drama, Romance",111,"UK, France, Belgium",Agnieszka Holland,Christopher Hampton,FIT Productions,"Leonardo DiCaprio, David Thewlis, Romane Bohri...",8000000,340139,0
2337,tt0117509,Romeo + Giulietta di William Shakespeare,1996,1997-02-28,"Drama, Romance",120,"USA, Mexico, Australia, Canada",Baz Luhrmann,"William Shakespeare, Craig Pearce",Bazmark Films,"Leonardo DiCaprio, Claire Danes, John Leguizam...",14500000,151842560,0
2355,tt0116999,La stanza di Marvin,1996,1997-04-24,Drama,98,USA,Jerry Zaks,"Scott McPherson, Scott McPherson",Scott Rudin Productions,"Meryl Streep, Leonardo DiCaprio, Diane Keaton,...",23000000,12803305,0
2535,tt0120338,Titanic,1997,1998-01-16,"Drama, Romance",194,"USA, Mexico, Australia, Canada",James Cameron,James Cameron,Twentieth Century Fox,"Leonardo DiCaprio, Kate Winslet, Billy Zane, K...",200000000,2195169869,0
2753,tt0120744,La maschera di ferro,1998,1998-03-27,"Action, Adventure, Drama",132,"France, USA",Randall Wallace,"Alexandre Dumas, Randall Wallace",United Artists,"Leonardo DiCaprio, Jeremy Irons, John Malkovic...",35000000,182968902,2


In [75]:
df['DirectorsOnes'] = 1

In [76]:
df['nbsuccessDirectors'] = df.groupby(by  ='director').cumsum()['DirectorsOnes']

In [79]:
df['nbsuccessProdCompa'] = df.groupby(by  ='production_company').cumsum()['DirectorsOnes']

In [82]:
df['nbsuccessWriter'] = df.groupby(by  ='writer').cumsum()['DirectorsOnes']

In [83]:
df.sort_values('nbsuccessWriter', ascending=False)

,imdb_title_id,title,year,date_published,genre,duration,country,director,writer,production_company,actors,budget,worlwide_gross_income,shifted,nbsuccessDirectors,DirectorsOnes,nbsuccessProdCompa,nbsuccessWriter
10952,tt5825380,La ruota delle meraviglie,2017,2017-12-14,Drama,101,USA,Woody Allen,Woody Allen,Amazon Studios,"Justin Timberlake, Juno Temple, Robert C. Kirk...",25000000,15889124,0,42,1,2,35
10737,tt4513674,Café Society,2016,2016-09-29,"Comedy, Drama, Romance",96,USA,Woody Allen,Woody Allen,Gravier Productions,"Steve Carell, Sheryl Lee, Todd Weeks, Paul Sch...",30000000,43763247,0,41,1,4,34
10109,tt3715320,Irrational Man,2015,2015-12-16,"Comedy, Drama, Romance",95,USA,Woody Allen,Woody Allen,Gravier Productions,"Joaquin Phoenix, Emma Stone, Joe Stapleton, Na...",11000000,27391084,4,40,1,3,33
9452,tt2870756,Magic in the Moonlight,2014,2014-12-04,"Comedy, Drama, Romance",97,"USA, France",Woody Allen,Woody Allen,Gravier Productions,"Colin Firth, Antonia Clarke, Natasha Andrews, ...",16800000,51029361,0,39,1,2,32
9001,tt2334873,Blue Jasmine,2013,2013-12-05,Drama,98,USA,Woody Allen,Woody Allen,Gravier Productions,"Cate Blanchett, Joy Carlin, Richard Conti, Gle...",18000000,99104804,1,38,1,1,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4309,tt0241025,La fiera della vanità,2004,2005-03-11,Drama,141,"USA, UK, India",Mira Nair,"Matthew Faulk, Mark Skeet",Focus Features,"Gabriel Byrne, Angelica Mandy, Roger Lloyd Pac...",23000000,19463185,0,4,1,5,1
4310,tt0196811,Olga,2004,2004-08-20,"Biography, Drama, History",141,Brazil,Jayme Monjardim,"Rita Buzzar, Fernando Morais",Europa Filmes,"Camila Morgado, Caco Ciocler, Luís Melo, Elian...",2156876,60820,0,1,1,1,1
4311,tt0368566,Albert est méchant,2004,2004-01-14,Comedy,85,France,Hervé Palud,"Jean-Marie Pallardy, Hervé Palud",Gaumont,"Christian Clavier, Michel Serrault, Arielle Do...",10042251,2753995,0,1,1,15,1
4313,tt0347368,L.A. Twister,2004,2004-06-30,Comedy,92,USA,Sven Pape,Geoffrey Saville-Read,L.A. Twister LLC,"Zack Ward, Tony Daly, Jennifer Aspen, Manousch...",450000,54000,0,1,1,1,1
